In [49]:
import pandas as pd

In [50]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['Дата'] = portfolio['Дата'].astype('datetime64[ns]')
transactions = pd.read_csv('transactions.csv').drop_duplicates(['Дата заключения', 'Время заключения', 'Статус'])
transactions['Дата заключения'] = pd.to_datetime(transactions['Дата заключения'] + ' ' + transactions['Время заключения'], dayfirst=True)
transactions['Дата расчетов'] = pd.to_datetime(transactions['Дата расчетов'], dayfirst=True)
transactions = transactions.drop('Время заключения', axis=1).dropna(axis=1).sort_values('Дата заключения').reset_index(drop=True)

In [51]:
transactions

,Дата заключения,Дата расчетов,Наименование,Код,Валюта,Вид,Количество,Цена,Сумма,НКД,Комиссия Брокера,Комиссия Биржи,Номер сделки,Статус
0,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,О
1,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,И
2,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.000,1633.0,0.0,0.98,0.15,3943134134,ЗО
3,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.000,5615.0,0.0,3.37,0.52,3943142919,И
4,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.000,5615.0,0.0,3.37,0.52,3943142919,О
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,2023-06-23 10:33:59,2023-06-27,МТС-ао,MTSS,RUB,Покупка,10,343.350,3433.5,0.0,2.06,1.03,7792729150,О
263,2023-07-18 16:07:57,2023-07-20,СевСт-ао,CHMF,RUB,Покупка,1,1275.600,1275.6,0.0,0.77,0.38,7908972071,И
264,2023-07-18 17:52:50,2023-07-20,Сбербанк,SBER,RUB,Покупка,10,246.210,2462.1,0.0,1.48,0.73,7909781928,И
265,2023-07-18 17:55:20,2023-07-20,Системаао,AFKS,RUB,Покупка,100,17.744,1774.4,0.0,1.06,0.54,7909796018,И


In [52]:
transactions_executed = transactions[transactions['Статус'] == 'И']

In [53]:
ticker_list = transactions_executed['Код'].unique()
share_split_dict = {
    'FXGD': ['2022-02-17', 10],
    'SBMX': ['2021-06-09', 100],
    'FXUS': ['2022-01-24', 100],
    'FXRL': ['2021-11-24', 100],
    'FXRU': ['2022-02-17', 10],
    'FXDE': ['2021-12-15', 100]
}

In [54]:
def fix_split():
    # is_split = round(transactions_executed.loc[transactions_executed['Код'] == ticker, 'Цена'].head(1).values[0] /\
            #    transactions_executed.loc[transactions_executed['Код'] == ticker, 'Цена'].tail(1).values[0])
    for ticker in ticker_list:
        if ticker in share_split_dict:
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] = \
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] * \
            share_split_dict[ticker][1]
fix_split()

In [55]:
def get_share_price_dict():
    share_price_dict = {}
    for ticker in ticker_list:
        if ticker in ['SBMX', 'AFKS']:
            round_numb = 3
        elif ticker in ['IRAO']:
            round_numb = 4
        else:
            round_numb = 2
        share = transactions_executed[transactions_executed['Код'] == ticker]
        share_price = ((share['Сумма'].sum() + share['Комиссия Брокера'].sum() + share['Комиссия Биржи'].sum()) / 
                            (share.loc[share['Вид'] == 'Покупка', 'Количество'].sum() - share.loc[share['Вид'] == 'Продажа', 'Количество'].sum())).round(round_numb)
        share_price_dict[ticker] = share_price
    return share_price_dict

In [56]:
share_price_dict = get_share_price_dict()

In [57]:
def get_share_amount():
    share_amount_dict = {}
    for ticker in ticker_list:
        share_amount_dict[ticker] = transactions_executed.loc[transactions_executed['Код'] == ticker, 'Количество'].sum()
    return share_amount_dict

In [58]:
share_amount_dict = get_share_amount()

In [60]:
data = {
    'amount': share_amount_dict,
    'price': share_price_dict
}
df = pd.DataFrame.from_dict(data)
df

,amount,price
SBSP,5,1724.6300
FXUS,350,57.1700
FXRL,192,39.8200
FXDM,92,76.6400
FXRU,43,94.3000
FXDE,124,30.8700
FXIM,36,90.1200
SBMX,830,15.7570
FXRW,1135,1.2800
VTBE,81,93.8900
